# DATA SCRAPPING PRACTICE

- Import BeautifulSoup and Requests for getting and parsing the webpage
- Import Pandas to put the data in a data frame

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Store URL in a variable

In [2]:
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

### I am scrapping English Premier League table data (on 12/07/2023) from [FBREF.com](https://fbref.com/en/comps/9/Premier-League-Stats)

Next step is to make a get request with 'requests' and then parse the data with 'BeautifulSoup' and store it in a variable

In [4]:
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202311303" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '_

### Next, i searched for the first table in the html, because that is where the data we need is stored

In [5]:
table = soup.find('table')

<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2023-202491_overall"> <caption>Regular season Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">Rk</th> <th aria-label="Squad" class="poptip sort_default_asc center" data-stat="team" scope="col">Squad</th> <th aria-label="Mat

### Next, i searched for the 'th' Table head tags to get all the headers.

I also added a class to the search to narrow down our search results to only the ones we need

In [12]:
table_heads = table.find_all('th', class_ ="poptip")

### Next i used a for loop to iterate over the list we got

I also took only the inner text and stripped it to remove any new line characters or extra space

In [13]:
table_heads_columns = [head.text.strip() for head in table_heads]

['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'xG', 'xGA', 'xGD', 'xGD/90', 'Last 5', 'Attendance', 'Top Team Scorer', 'Goalkeeper', 'Notes']


### Next i created the Data Frame and set the columns to the information we got from our for loop

I also set the index name to 'Rk' and i dropped the 'Rk' row

In [30]:
df = pd.DataFrame(columns=table_heads_columns)
df.index.name = 'Rk'
df = df.drop(columns=['Rk'])

df

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,,


### Next, i searched for all the 'tr' Table row tags because they were wrapping all the data we needed for our rows

In [25]:
table_data = table.find_all('tr')

### Next step is to iterate over the list and extract all the data we need

To do that, i used a for loop that started on the item on index 1 because index 0 contains our header data. I also searched for all the 'td' tags inside our table data because they served as an inner wrapper for our data. Then i took the text within them and stripped it to remove unneccessary space and new line characters

After that, i used the .loc and length of the dataframe to select an index and insert each column of data.

In [31]:
for data in table_data[1:]:
    row_data = data.find_all('td')
    ind_data = [rdata.text.strip() for rdata in row_data]
    length = len(df)
    df.loc[length] = ind_data
    
df

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,,
0,Arsenal,15,11,3,1,33,14,+19,36,2.40,27.6,11.5,+16.1,+1.07,L W W W W,"60,147","Eddie Nketiah, Bukayo Saka - 5",David Raya,
1,Liverpool,15,10,4,1,34,14,+20,34,2.27,30.9,18.0,+12.9,+0.86,D W D W W,"50,573",Mohamed Salah - 10,Alisson,
2,Aston Villa,15,10,2,3,34,20,+14,32,2.13,28.2,20.8,+7.4,+0.49,L W W D W,"35,467",Ollie Watkins - 8,Emiliano Martínez,
3,Manchester City,15,9,3,3,36,17,+19,30,2.00,29.0,14.4,+14.6,+0.97,W D D D L,"45,700",Erling Haaland - 14,Ederson,
4,Tottenham,14,8,3,3,28,20,+8,27,1.93,22.0,22.9,-0.9,-0.07,W L L L D,"61,718",Son Heung-min - 9,Guglielmo Vicario,
5,Manchester Utd,15,9,0,6,18,18,0,27,1.80,24.3,25.1,-0.8,-0.05,W W W L W,"73,517",Scott McTominay - 5,André Onana,
6,Newcastle Utd,14,8,2,4,32,14,+18,26,1.86,28.1,14.2,+13.9,+0.99,D W L W W,"52,075","Callum Wilson, Alexander Isak - 7",Nick Pope,
7,Brighton,15,7,4,4,32,27,+5,25,1.67,24.9,22.4,+2.5,+0.17,D D W L W,"27,661",Evan Ferguson - 6,Jason Steele,
8,West Ham,14,6,3,5,24,24,0,21,1.50,19.6,24.4,-4.8,-0.34,L L W W D,"62,459",Jarrod Bowen - 8,Alphonse Areola,


### After doing all that, we have all the data we need

We can export the data if we want

In [32]:
df.to_csv(r'C:\Users\Owner\OneDrive\Desktop\Excel\PL_12-07-2023.csv')